In [ ]:
%load_ext tensorboard
%load_ext dotenv

%dotenv

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
import os
import openai
from pydantic import BaseModel

MODEL = "gpt-4o-mini"

openai.api_key = os.getenv("OPENAI_API_KEY")

In [47]:
x="Hospital Management System"

In [48]:
generate_personas_prompt = "Generate a list of {count} distinct but relevant personas for the following topic. Each persona should have a unique perspective, background, or expertise that would contribute meaningfully to a discussion on this topic. The personas should vary in profession, experience, or worldview to ensure diverse viewpoints."


class Persona(BaseModel):
    name: str
    description: str

class Personas(BaseModel):
    items: list[Persona]

def generate_personas(topic: str, count: int):
    prompt = generate_personas_prompt.format(count=count, topic=topic)

    response = openai.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": x}
        ],
        response_format=Personas
    )

    # todo make sure we have the right number of personas, else generate more

    return response.choices[0].message.parsed

personas = generate_personas("The future of work", 4)

print(personas.model_dump_json(indent=True))

{
 "items": [
  {
   "name": "Dr. Sarah Thompson",
   "description": "A hospital administrator with over 15 years of experience in healthcare management. Dr. Thompson specializes in optimizing hospital operations and implementing advanced management systems to improve patient care and administrative efficiency. Her perspective focuses on the integration of technology in hospitals and the potential impact on patient outcomes."
  },
  {
   "name": "Mark Rodriguez",
   "description": "An IT specialist who has spent a decade developing software solutions for healthcare facilities. Mark has hands-on experience with Electronic Health Records (EHR) systems, patient management tools, and interoperability issues between different health IT systems. He provides insights into the technical challenges and innovations in hospital management systems."
  },
  {
   "name": "Emily Chen",
   "description": "A nurse with a background in patient advocacy and clinical education. Emily has worked on the fro

In [49]:
generate_cqs_prompt = "You are {name}, {description}. You are part of an interdisciplinary research team that is tasked with creating an ontology. As a first step, you are asked to come up with a set of {count} competency questions related to the topic below. These questions should help define key concepts, relationships, and constraints necessary for building an ontology. Always remember that good competency questions should be clear and unambiguous, relevant to the domain requirements, answerable by the ontology and useful for verification purposes. Keep them EXTREMELY SHORT but INFORMATIVE!"

class CompetencyQuestion(BaseModel):
    question: str

class CompetencyQuestions(BaseModel):
    items: list[CompetencyQuestion]

def generate_competency_questions(persona: Persona, topic: str, count: int):
    prompt = generate_cqs_prompt.format(count=count, name=persona.name, description=persona.description, topic=topic)

    response = openai.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": x}
        ],
        response_format=CompetencyQuestions
    )

    # todo make sure we have the right number of questions, else generate more

    return response.choices[0].message.parsed

cqs_per_persona = dict()

for persona in personas.items:
    print(persona.name)

    cqs = generate_competency_questions(persona, x, 3).items

    for q in cqs:
        print(" - ", q.question)

    cqs_per_persona[persona.name] = cqs

Dr. Sarah Thompson
 -  What are the key roles within hospital management?
 -  How are patient records linked to healthcare providers in the system?
 -  What metrics are used to assess the efficiency of hospital operations?
Mark Rodriguez
 -  What types of entities are involved in patient management?
 -  How are patient records linked to different healthcare providers?
 -  What relationships exist between hospital departments and their services?
Emily Chen
 -  What are the key user roles within the hospital management system?
 -  How is patient data linked to clinical workflows in the system?
 -  What functionalities enhance communication between nursing staff and other health professionals?
Dr. Alan Patel
 -  What are the core functionalities of a hospital management system?
 -  How do regulatory requirements influence the adoption of new technologies in hospitals?
 -  What relationships exist between hospital performance metrics and patient outcomes?


# Thoughts
* maybe immediately jumping into CQs is not a good idea, consider evaluating the scope/space of the problem? (simulated discussion?)

In [51]:
# consider re-emphasizing the requirements of competency questions

give_feedback_prompt = "You are {name}, {description}. You have been asked to review the competency questions created by your team members listed below. Please provide constructive feedback BUT ONLY WHERE STRICTLY necessary. Your feedback should be specific, actionable, and focused on improving the clarity, relevance, and usefulness of the questions. Keep it EXTREMELY SHORT such that your colleagues know what you mean."

class Comment(BaseModel):
    question_id: int
    comment: str
    referenced_question_ids: list[int] | None = None

class Feedback(BaseModel):
    comments: list[Comment]

def collect_feedback(persona: Persona, competency_questions: list[CompetencyQuestion]):
    # consider adding information about whose competency question it is

    prompt = give_feedback_prompt.format(name=persona.name, description=persona.description)

    questions_str = "\n".join([f"{i}. {cq.question}" for i, cq in enumerate(competency_questions)])

    response = openai.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"<questions>{questions_str}</questions>"},
            {"role": "user", "content": f"<topic>{x}</topic>"}
        ],
        response_format=Feedback
    )

    return response.choices[0].message.parsed

def get_cqs_except_of(persona: Persona):
    return [cq for p, cqs in cqs_per_persona.items() if p != persona.name for cq in cqs]

total_feedback = dict()

for persona in personas.items:
    print(persona.name)
    cqs_to_comment_on = get_cqs_except_of(persona)
    feedback = collect_feedback(persona, cqs_to_comment_on)

    total_feedback[persona.name] = feedback

    for comment in feedback.comments:
        print(" - ", cqs_to_comment_on[comment.question_id].question,"\"")
        print(" |-> \"", comment.comment)





Dr. Sarah Thompson
 -  What types of entities are involved in patient management? "
 |-> " Consider specifying the types of entities, such as internal and external stakeholders, to clarify the scope.
 -  How are patient records linked to different healthcare providers? "
 |-> " Refine the question to focus on specific mechanisms or technologies that link patient records.
 -  What relationships exist between hospital departments and their services? "
 |-> " Clarify which hospital departments you are referring to for better context.
 -  What are the key user roles within the hospital management system? "
 |-> " Add examples of user roles to provide context and enhance understanding.
 -  How is patient data linked to clinical workflows in the system? "
 |-> " Emphasize how this linking impacts patient care to improve relevance.
 -  What functionalities enhance communication between nursing staff and other health professionals? "
 |-> " Consider detailing specific functionalities that impr

In [46]:
# todo now, collect feedback per question, and prompt personas with their question and the feedback and let them revise it.
